In [ ]:
!pip install -q condacolab

In [ ]:
import condacolab

In [ ]:
condacolab.install()

✨🍰✨ Everything looks OK!


In [ ]:
# Install PyTorch
!conda install pytorch torchvision torchaudio
# Install transformers library
!pip install transformers
# Install other required packages
!pip install langchain sentence_transformers huggingface-hub# Install PyTorch
!conda install pytorch torchvision torchaudio
# Install transformers library
!pip install transformers
# Install other required packages
!pip install langchain sentence_transformers huggingface-hub

Channels:
 - conda-forge
Platform: linux-64
Solving environment: | / - \ | failed

PackagesNotFoundError: The following packages are not available from current channels:

  - torchaudio

Current channels:

  - https://conda.anaconda.org/conda-forge

To search for alternate channels that may provide the conda package you're
looking for, navigate to

    https://anaconda.org

and use the search bar at the top of the page.


ERROR: Invalid requirement: 'huggingface-hub#'
Channels:
 - conda-forge
Platform: linux-64
Solving environment: \ | / - failed

PackagesNotFoundError: The following packages are not available from current channels:

  - torchaudio

Current channels:

  - https://conda.anaconda.org/conda-forge

To search for alternate channels that may provide the conda package you're
looking for, navigate to

    https://anaconda.org

and use the search bar at the top of the page.




In [ ]:
pip install langchain-community

In [ ]:
!pip install faiss-gpu
!pip install faiss-cpu

In [ ]:
!pip install bitsandbytes
!pip install --upgrade transformers

In [ ]:
pip install -U transformers

In [ ]:
pip install bitsandbytes accelerate

In [ ]:
pip install pdfplumber

In [ ]:
pip install PyMuPDF

In [ ]:
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)

Using device: cuda


In [ ]:
import os
#import pdfplumber  # or import fitz for PyMuPDF
import fitz

def pdf_to_txt(pdf_path, txt_path):
    try:
        pdf_document = fitz.open(pdf_path)
        with open(txt_path, 'w', encoding='utf-8') as txt_file:
            for page_num in range(len(pdf_document)):
                page = pdf_document.load_page(page_num)
                page_text = page.get_text()
                if page_text:
                    txt_file.write(page_text + '\n')
                else:
                    print(f"Warning: No text extracted from page {page_num} in {pdf_path}")
        print(f"Converted {pdf_path} to {txt_path}")
    except Exception as e:
        print(f"Error converting {pdf_path} to text: {e}")

def convert_all_pdfs_in_folder(folder_path):
    for filename in os.listdir(folder_path):
        if filename.lower().endswith('.pdf'):
            pdf_path = os.path.join(folder_path, filename)
            txt_path = os.path.join(folder_path, filename.replace('.pdf', '.txt'))
            pdf_to_txt(pdf_path, txt_path)

# Convert PDFs to text files
folder_path = r'/content/documents'
convert_all_pdfs_in_folder(folder_path)



Converted /content/documents/Manual.pdf to /content/documents/Manual.txt


In [ ]:
#from langchain.document_loaders import DirectoryLoader
from langchain.document_loaders import DirectoryLoader, TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
folder_path = r'/content/documents'
text_loader_kwargs = {'autodetect_encoding': True}
mixed_loader = DirectoryLoader(
    path=folder_path,
    glob='*.txt',  # Use forward slash for paths
    loader_cls=TextLoader,
    loader_kwargs=text_loader_kwargs
)
doc = mixed_loader.load()

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
print(f"Number of doc loaded: {len(doc)}")

docs = text_splitter.split_documents(doc)
print(f"Number of documents loaded: {len(docs)}")


Number of doc loaded: 1
Number of documents loaded: 43


In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
encoder = HuggingFaceEmbeddings()
db = FAISS.from_documents(documents=docs, embedding=encoder)
retriever = db.as_retriever(search_kwargs={"k": 10})

<ipython-input-58-b33ff49ad885>:3: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  encoder = HuggingFaceEmbeddings()


In [ ]:

from huggingface_hub import login

login(token="")


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
pip install -U huggingface_hub

In [ ]:
#from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
#quantization_config = BitsAndBytesConfig(load_in_4bit=True)
#model = AutoModelForCausalLM.from_pretrained("google/gemma-1.1-7b-it", quantization_config=quantization_config)
#tokenizer = AutoTokenizer.from_pretrained("google/gemma-1.1-7b-it", quantization_config=quantization_config)
#import torch
#torch.set_grad_enabled(False)

from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

quantization_config = BitsAndBytesConfig(load_in_4bit=True)

model = "google/gemma-1.1-7b-it"
tokenizer = AutoTokenizer.from_pretrained(model)
model = AutoModelForCausalLM.from_pretrained(
    model,
    quantization_config=quantization_config,
    device_map="auto"
)

ValueError: Some modules are dispatched on the CPU or the disk. Make sure you have enough GPU RAM to fit the quantized model. If you want to dispatch the model on the CPU or the disk while keeping these modules in 32-bit, you need to set `load_in_8bit_fp32_cpu_offload=True` and pass a custom `device_map` to `from_pretrained`. Check https://huggingface.co/docs/transformers/main/en/main_classes/quantization#offload-between-cpu-and-gpu for more details. 

In [ ]:
from langchain.chains import RetrievalQA
from langchain.llms import HuggingFacePipeline
from transformers import pipeline

# Define the query
query = "How to make a SOS call?"

# Retrieve relevant documents
retrieved_docs = retriever.get_relevant_documents(query)

# Combine the retrieved documents into a single context string
context = " ".join([doc.page_content for doc in retrieved_docs])

# Create a Hugging Face pipeline
huggingface_pipeline = pipeline(
    task="text-generation",
    model=model,
    tokenizer=tokenizer,
    model_kwargs={
        "temperature": 0.7,
        "max_length": 512,
        "max_new_tokens": 200
    }
)

# Define a function to generate text with context
def generate_with_context(context, query):
    # Combine query with context for generation
    input_text = f"Context: {context}\nQuestion: {query}"
    input_ids = tokenizer(input_text, return_tensors="pt").to("cuda")
    outputs = model.generate(**input_ids, max_new_tokens=200)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Generate response based on query and context
response = generate_with_context(context, query)
print(response)


Context: To be able to make SOS calls, you need to add your ICE (In
Case of Emergency) details.
To add your personal details, select Menu > Settings >
Phone settings > ICE information.
To select the contacts to whom the SOS call is made,
select Menu > Contacts > Set ICE contact. Select each ICE
contact and then the contact from your contacts list.

Note that you cannot use official emergency call numbers
as your ICE contacts.
Before you can make SOS calls, you also need to switch
on the accessibility mode. Select Menu > Settings >
Phone settings > Accessibility mode > SOS call > On.

Make an SOS call Once you have the accessibility mode switched on and
added an ICE contact, you can make SOS calls.
To make the call, press and hold the number 2 key for
three seconds.
The phone calls your first ICE contact. If the contact
doesn’t answer in 25 seconds, the phone calls the next
contact, and keeps calling your contacts 10 times until
one of them answers the call, or you press
.
Note: When t